## Linear Regression: Predict life expectancy from birth rate

### Dataset Description
- Name: Birth rate - life expectancy in 2010
- X = birth rate. Type: float.
- Y = life expectancy. Type: foat.
- Number of datapoints: 190


### Approach 1
assume that the relationship between the birth rate and the life expectancy is linear, which means that we can find w and b such that Y = wX + b

In [1]:
import os
print(os.getcwd())



/Users/surkpark/cs20si_study


In [2]:
import sys
sys.path.append(os.getcwd()+"/examples")

In [6]:
import tensorflow as tf

import utils

DATA_FILE = "./exapmles/data/birth_life_2010.txt"

In [7]:
# Step 1: read in data from the .txt file
# data is a numpy array of shape (190, 2), each row is a datapoint
data, n_samples = utils.read_birth_life_data(DATA_FILE)

In [8]:
# Step 2: create placeholders for X (birth rate) and Y (life expectancy)
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')


In [9]:
# Step 3: create weight and bias, initialized to 0
w = tf.get_variable('weights', initializer=tf.constant(0.0))
b = tf.get_variable('bias', initializer=tf.constant(0.0))

In [10]:
# Step 4: construct model to predict Y (life expectancy from birth rate)
Y_predicted = w * X + b 


In [11]:
# Step 5: use the square error as the loss function
loss = tf.square(Y - Y_predicted, name='loss')

In [12]:
# Step 6: using gradient descent with learning rate of 0.01 to minimize loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

In [13]:
with tf.Session() as sess:
    # Step 7: initialize the necessary variables, in this case, w and b
    sess.run(tf.global_variables_initializer()) 
    
    # Step 8: train the model
    for i in range(100): # run 100 epochs
        for x, y in data:
        # Session runs train_op to minimize loss
            sess.run(optimizer, feed_dict={X: x, Y:y}) 
    
    # Step 9: output the values of w and b
    w_out, b_out = sess.run([w, b]) 